In [1]:
import json, os
import ccd
import numpy as np
import pandas as pd
from datetime import datetime
from test.shared import snap, rainbow

from numba import jit
# detect results for x, y: -2094435, 1681005 took 2.455234 seconds to generate  VERIFIED 2.6 seconds locally
# detect results for x, y: -2033595, 1685955 took 1.879693 seconds to generate
# detect results for x, y: -2098875, 1694895 took 0.986887 seconds to generate  VERIFIED 1.8 seconds locally
pixl_x, pixl_y = -2094435, 1681005
pixl_x, pixl_y = -2098875, 1694895

inputs_url = "https://lcmaphost.com?x={x}&y={y}&acquired=1982-01-01/2015-12-31\
&ubid=LANDSAT_4/TM/SRB1&ubid=LANDSAT_4/TM/SRB2&ubid=LANDSAT_4/TM/SRB3&ubid=LANDSAT_4/TM/SRB4\
&ubid=LANDSAT_4/TM/SRB5&ubid=LANDSAT_4/TM/BTB6&ubid=LANDSAT_4/TM/SRB7&ubid=LANDSAT_4/TM/PIXELQA\
&ubid=LANDSAT_5/TM/SRB1&ubid=LANDSAT_5/TM/SRB2&ubid=LANDSAT_5/TM/SRB3&ubid=LANDSAT_5/TM/SRB4\
&ubid=LANDSAT_5/TM/SRB5&ubid=LANDSAT_5/TM/BTB6&ubid=LANDSAT_5/TM/SRB7&ubid=LANDSAT_5/TM/PIXELQA\
&ubid=LANDSAT_7/ETM/SRB1&ubid=LANDSAT_7/ETM/SRB2&ubid=LANDSAT_7/ETM/SRB3&ubid=LANDSAT_7/ETM/SRB4\
&ubid=LANDSAT_7/ETM/SRB5&ubid=LANDSAT_7/ETM/BTB6&ubid=LANDSAT_7/ETM/SRB7&ubid=LANDSAT_7/ETM/PIXELQA\
&ubid=LANDSAT_8/OLI_TIRS/SRB2&ubid=LANDSAT_8/OLI_TIRS/SRB3&ubid=LANDSAT_8/OLI_TIRS/SRB4&ubid=LANDSAT_8/OLI_TIRS/SRB5\
&ubid=LANDSAT_8/OLI_TIRS/SRB6&ubid=LANDSAT_8/OLI_TIRS/SRB7&ubid=LANDSAT_8/OLI_TIRS/BTB10\
&ubid=LANDSAT_8/OLI_TIRS/PIXELQA".format(x=pixl_x, y=pixl_y)

dates           = [i.split('=')[1] for i in inputs_url.split('&') if 'acquired=' in i][0]
chips_url       = inputs_url.split('?')[0]
specs_url       = chips_url.replace('/chips', '/chip-specs')
querystr_list   = inputs_url.split('?')[1].split('&')
requested_ubids = [i.replace('ubid=', '') for i in querystr_list if 'ubid=' in i]

# get chip id
chip_x, chip_y = snap(pixl_x, pixl_y)
xindex = int((pixl_x - chip_x) / 30)
yindex = int((chip_y - pixl_y) / 30)
_start = datetime.now()
rbow = rainbow(-2094585, 1682805, dates, specs_url, chips_url, requested_ubids)
print("took: {}".format((datetime.now() - _start).total_seconds()))

@jit
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()

took: 15.887416


In [2]:
row, col = yindex, xindex
rainbow_date_array = np.array(rbow['t'].values)
_start = datetime.now()
result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                     np.array(rbow['blue'].values[:, row, col]),
                     np.array(rbow['green'].values[:, row, col]),
                     np.array(rbow['red'].values[:, row, col]),
                     np.array(rbow['nir'].values[:, row, col]),
                     np.array(rbow['swir1'].values[:, row, col]),
                     np.array(rbow['swir2'].values[:, row, col]),
                     np.array(rbow['thermal'].values[:, row, col]),
                     np.array(rbow['cfmask'].values[:, row, col], dtype=int),
                     params={})
print("took: {}".format((datetime.now() - _start).total_seconds()))

AttributeError: '_DisableJitWrapper' object has no attribute '__name__'

In [2]:
rainbow_date_array = np.array(rbow['t'].values)
_st = datetime.now()
_results = []
for _x in range(0, 100):
    for _y in range(0, 100):
        row = _y
        col = _x        
        result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                              np.array(rbow['blue'].values[:, row, col]),
                              np.array(rbow['green'].values[:, row, col]),
                              np.array(rbow['red'].values[:, row, col]),
                              np.array(rbow['nir'].values[:, row, col]),
                              np.array(rbow['swir1'].values[:, row, col]),
                              np.array(rbow['swir2'].values[:, row, col]),
                              np.array(rbow['thermal'].values[:, row, col]),
                              np.array(rbow['cfmask'].values[:, row, col], dtype=int),
                              params={})
        if result:
            _results.append((_x, _y))

        
print("took: {}".format((datetime.now() - _st).total_seconds()))

/home/caustin/miniconda2/envs/ccd35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


took: 15165.333728


In [4]:
15165 / 60 / 60

4.2125